In [27]:
import requests
import pandas as pd
import os
import json


url = "https://services5.arcgis.com/SaBe5HMtmnbqSWlu/arcgis/rest/services/UkrainianCoTTimelapse_FEB_2022_to_DEC_2024_view/FeatureServer/75/query"


In [11]:


params = {
    "f": "geojson",  # Change format to GeoJSON
    "where": "1=1",  # No filter (fetch all data)
    "outFields": "OBJECTID,datetime",  # Keep OBJECTID and datetime
    "orderByFields": "OBJECTID",
    "resultRecordCount": 100,  # Fetch max records per request
    "resultOffset": 0,  # Start at first record
    "maxRecordCountFactor": 4,  # Increase max records
    "spatialRel": "esriSpatialRelIntersects",  # Keep standard spatial relation
    "geometryType": "esriGeometryEnvelope",  # Required, but not filtering by bbox
    "resultType": "tile",  # Standard result type
}

headers = {
    "Referer": "https://storymaps.arcgis.com/stories/733fe90805894bfc8562d90b106aa895",
    "User-Agent": "Mozilla/5.0",
}

response = requests.get(url, params=params, headers=headers)

response.json()

{'type': 'FeatureCollection',
 'properties': {'exceededTransferLimit': True},
 'features': [{'type': 'Feature',
   'id': 1,
   'geometry': {'type': 'Polygon',
    'coordinates': [[[35.3720730183027, 47.5320031143062],
      [35.3726772808584, 47.531874614978],
      [35.3730742131448, 47.5317381061591],
      [35.3734657528445, 47.5314933274473],
      [35.3737847859244, 47.5312093820067],
      [35.3740530632924, 47.5308764784979],
      [35.3742850873505, 47.53057784176],
      [35.3745260613238, 47.5301921728328],
      [35.3748447629253, 47.5292296885844],
      [35.375499673395, 47.5272517905141],
      [35.3768156001834, 47.5214029238012],
      [35.3767877560029, 47.5212901070182],
      [35.3768155992851, 47.5211334177853],
      [35.3769501732029, 47.5209955299996],
      [35.377312130482, 47.5207730296208],
      [35.3781381358773, 47.5203562289483],
      [35.3793028968646, 47.5197043851918],
      [35.3807455058709, 47.5189493090817],
      [35.3837545674253, 47.51741659841

In [15]:
ts = 1716854400000
dt = pd.to_datetime(ts, unit='ms')
dt


Timestamp('2024-05-28 00:00:00')

# What datetimes?



In [51]:
import requests

url = "https://services5.arcgis.com/SaBe5HMtmnbqSWlu/arcgis/rest/services/UkrainianCoTTimelapse_FEB_2022_to_DEC_2024_view/FeatureServer/75/query"

params = {
    "f": "json",  # JSON for easy parsing
    "where": "1=1",  # No filter (fetch all dates)
    "outFields": "datetime",  # Request only datetime
    "returnGeometry": "false",  # No geometry
    "returnDistinctValues": "true",  # Get unique dates only
    "orderByFields": "datetime ASC",  # Sort from earliest to latest
}

headers = {
    "Referer": "https://storymaps.arcgis.com/stories/733fe90805894bfc8562d90b106aa895",
    "User-Agent": "Mozilla/5.0",
}

response = requests.get(url, params=params, headers=headers)

if response.status_code == 200:
    data = response.json()
    dates = [feature["attributes"]["datetime"] for feature in data.get("features", [])]
    print("Available dates:", dates)
else:
    print(f"Request failed: {response.status_code} - {response.text}")

Available dates: [None, 1646179200000, 1646265600000, 1646352000000, 1646438400000, 1646524800000, 1646611200000, 1646697600000, 1646784000000, 1646870400000, 1646956800000, 1647043200000, 1647129600000, 1647216000000, 1647302400000, 1647388800000, 1647475200000, 1647561600000, 1647648000000, 1647734400000, 1647820800000, 1647907200000, 1647993600000, 1648080000000, 1648166400000, 1648252800000, 1648339200000, 1648425600000, 1648512000000, 1648598400000, 1648684800000, 1648771200000, 1648857600000, 1648944000000, 1649030400000, 1649116800000, 1649203200000, 1649289600000, 1649376000000, 1649462400000, 1649548800000, 1649635200000, 1649721600000, 1649808000000, 1649894400000, 1649980800000, 1650067200000, 1650153600000, 1650240000000, 1650326400000, 1650412800000, 1650499200000, 1650585600000, 1650672000000, 1650758400000, 1650844800000, 1650931200000, 1651017600000, 1651104000000, 1651190400000, 1651276800000, 1651363200000, 1651449600000, 1651536000000, 1651622400000, 1651708800000, 1

# Requesting

In [53]:
url

'https://services5.arcgis.com/SaBe5HMtmnbqSWlu/arcgis/rest/services/UkrainianCoTTimelapse_FEB_2022_to_DEC_2024_view/FeatureServer/75/query'

In [70]:
def make_map_req(target_ts, layer_root):
    url = f"{layer_root}query"
    # print(url)

    # Convert to a formatted timestamp string
    ts_start = target_ts.strftime("%Y-%m-%d %H:%M:%S")
    ts_end = (target_ts + pd.Timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")

    params = {
        "f": "geojson",
        "where": "datetime BETWEEN TIMESTAMP '{}' AND TIMESTAMP '{}'".format(ts_start, ts_end),  # Proper ArcGIS format
        "outFields": "OBJECTID,datetime",
        "orderByFields": "OBJECTID",
        "resultRecordCount": 8000,
        "resultOffset": 0,
        "maxRecordCountFactor": 4,
        "returnGeometry": "true",
        "spatialRel": "esriSpatialRelIntersects",
        "geometryType": "esriGeometryEnvelope",
        "resultType": "tile",
    }

    headers = {
        "Referer": "https://storymaps.arcgis.com/stories/733fe90805894bfc8562d90b106aa895",
        "User-Agent": "Mozilla/5.0",
    }

    response = requests.get(url, params=params, headers=headers)

    return response.json()

dates = [pd.to_datetime(date, unit='ms') for date in dates]
dates

target_timestamps = pd.date_range(start=pd.Timestamp("2022-03-02"), end=dates[-1], freq="M")

all_map_data = []

layer_roots = ['https://services5.arcgis.com/SaBe5HMtmnbqSWlu/arcgis/rest/services/Ukraine_Timelapse_FEB_to_JAN/FeatureServer/8/',
 'https://services5.arcgis.com/SaBe5HMtmnbqSWlu/arcgis/rest/services/UkrainianCoTTimelapse_FEB_2022_to_DEC_2024_view/FeatureServer/75/',
 'https://services5.arcgis.com/SaBe5HMtmnbqSWlu/arcgis/rest/services/UkrainianCoTTimelapse_FEB_2022_to_DEC_2024_view/FeatureServer/0/',
 'https://services5.arcgis.com/SaBe5HMtmnbqSWlu/arcgis/rest/services/VIEW_Russian_controlled_Ukrainian_Territory_before_February_24_2022/FeatureServer/36/',
 'https://services5.arcgis.com/SaBe5HMtmnbqSWlu/arcgis/rest/services/UkrainianCoTTimelapse_FEB_2022_to_DEC_2024_view/FeatureServer/1/',
 'https://services5.arcgis.com/SaBe5HMtmnbqSWlu/arcgis/rest/services/UkrainianCoTTimelapse_FEB_2022_to_DEC_2024_view/FeatureServer/2/']


for target_ts in target_timestamps:
    print(f'\n{pd.to_datetime(target_ts).strftime("%Y-%m-%d")}')
    for layer_root in layer_roots:
        layer_id = layer_root.split('/')[-2]

        if not os.path.exists(f"geojson_dump/{layer_id}"):
            os.makedirs(f"geojson_dump/{layer_id}")

        # if os.path.exists(f"geojson_dump/{target_ts.strftime('%Y-%m-%d')}.geojson"):
        #     print("\tSkipping existing file")
        #     with open(f"geojson_dump/{target_ts.strftime('%Y-%m-%d')}.geojson", "r") as f:
        #         all_map_data.append(json.load(f))
        #     continue

        map_data = make_map_req(target_ts, layer_root)
        with open(f"geojson_dump/{layer_id}/{target_ts.strftime('%Y-%m-%d')}.geojson", "w") as f:
            json.dump(map_data, f)
        all_map_data.append(map_data)
        print(".", end="")
        


2022-03-31
......
2022-04-30
......
2022-05-31
......
2022-06-30
......
2022-07-31
......
2022-08-31
......
2022-09-30
......
2022-10-31
......
2022-11-30
......
2022-12-31
......
2023-01-31
......
2023-02-28
......
2023-03-31
......
2023-04-30
......
2023-05-31
......
2023-06-30
......
2023-07-31
......
2023-08-31
......
2023-09-30
......
2023-10-31
......
2023-11-30
......
2023-12-31
......
2024-01-31
......
2024-02-29
......
2024-03-31
......
2024-04-30
......
2024-05-31
......
2024-06-30
......
2024-07-31
......
2024-08-31
......
2024-09-30
......
2024-10-31
......
2024-11-30
......
2024-12-31
......

# Identifying layers

In [66]:
with open("map.har") as f:
    har = json.load(f)


query_urls = []
for entry in har["log"]["entries"]:
    if "query" in entry["request"]["url"]:
        query_urls += [entry["request"]["url"]]

#'https://services5.arcgis.com/SaBe5HMtmnbqSWlu/arcgis/rest/services/Ukraine_Timelapse_FEB_to_JAN/FeatureServer/8/query?f=pbf&geometry=-0.000002983957529067993%2C7514065.62854699%2C2504688.542845018%2C10018754.171394993&maxRecordCountFactor=4&resultOffset=0&resultRecordCount=8000&where=1%3D1&orderByFields=OBJECTID&outFields=ChangeDate%2CEndDate%2COBJECTID&quantizationParameters=%7B%22extent%22%3A%7B%22xmin%22%3A-0.000002983957529067993%2C%22ymin%22%3A7514065.62854699%2C%22xmax%22%3A2504688.542845018%2C%22ymax%22%3A10018754.171394993%7D%2C%22mode%22%3A%22view%22%2C%22originPosition%22%3A%22upperLeft%22%2C%22tolerance%22%3A4891.969810250004%7D&resultType=tile&spatialRel=esriSpatialRelIntersects&geometryType=esriGeometryEnvelope&defaultSR=102100'
url = "https://services5.arcgis.com/SaBe5HMtmnbqSWlu/arcgis/rest/services/Ukraine_Timelapse_FEB_to_JAN/FeatureServer/8/query?f=pbf&geometry=-0.000002983957529067993%2C7514065.62854699%2C2504688.542845018%2C10018754.171394993&maxRecordCountFactor=4&resultOffset=0&resultRecordCount=8000&where=1%3D1&orderByFields=OBJECTID&outFields=ChangeDate%2CEndDate%2COBJECTID&quantizationParameters=%7B%22extent%22%3A%7B%22xmin%22%3A-0.000002983957529067993%2C%22ymin%22%3A7514065.62854699%2C%22xmax%22%3A2504688.542845018%2C%22ymax%22%3A10018754.171394993%7D%2C%22mode%22%3A%22view%22%2C%22originPosition%22%3A%22upperLeft%22%2C%22tolerance%22%3A4891.969810250004%7D&resultType=tile&spatialRel=esriSpatialRelIntersects&geometryType=esriGeometryEnvelope&defaultSR=102100"
layer_ids = [url.split("FeatureServer/")[1].split("/")[0] for url in query_urls]
list(set(layer_ids))


url_bases = [url.split("query")[0] for url in query_urls]
list(set(url_bases))

['https://services5.arcgis.com/SaBe5HMtmnbqSWlu/arcgis/rest/services/Ukraine_Timelapse_FEB_to_JAN/FeatureServer/8/',
 'https://services5.arcgis.com/SaBe5HMtmnbqSWlu/arcgis/rest/services/UkrainianCoTTimelapse_FEB_2022_to_DEC_2024_view/FeatureServer/75/',
 'https://services5.arcgis.com/SaBe5HMtmnbqSWlu/arcgis/rest/services/UkrainianCoTTimelapse_FEB_2022_to_DEC_2024_view/FeatureServer/0/',
 'https://services5.arcgis.com/SaBe5HMtmnbqSWlu/arcgis/rest/services/VIEW_Russian_controlled_Ukrainian_Territory_before_February_24_2022/FeatureServer/36/',
 'https://services5.arcgis.com/SaBe5HMtmnbqSWlu/arcgis/rest/services/UkrainianCoTTimelapse_FEB_2022_to_DEC_2024_view/FeatureServer/1/',
 'https://services5.arcgis.com/SaBe5HMtmnbqSWlu/arcgis/rest/services/UkrainianCoTTimelapse_FEB_2022_to_DEC_2024_view/FeatureServer/2/']

In [64]:
query_urls

['https://services5.arcgis.com/SaBe5HMtmnbqSWlu/arcgis/rest/services/Ukraine_Timelapse_FEB_to_JAN/FeatureServer/8/query?f=pbf&geometry=-0.000002983957529067993%2C7514065.62854699%2C2504688.542845018%2C10018754.171394993&maxRecordCountFactor=4&resultOffset=0&resultRecordCount=8000&where=1%3D1&orderByFields=OBJECTID&outFields=ChangeDate%2CEndDate%2COBJECTID&quantizationParameters=%7B%22extent%22%3A%7B%22xmin%22%3A-0.000002983957529067993%2C%22ymin%22%3A7514065.62854699%2C%22xmax%22%3A2504688.542845018%2C%22ymax%22%3A10018754.171394993%7D%2C%22mode%22%3A%22view%22%2C%22originPosition%22%3A%22upperLeft%22%2C%22tolerance%22%3A4891.969810250004%7D&resultType=tile&spatialRel=esriSpatialRelIntersects&geometryType=esriGeometryEnvelope&defaultSR=102100',
 'https://services5.arcgis.com/SaBe5HMtmnbqSWlu/arcgis/rest/services/Ukraine_Timelapse_FEB_to_JAN/FeatureServer/8/query?f=pbf&geometry=2504688.542845018%2C7514065.62854699%2C5009377.08569302%2C10018754.171394993&maxRecordCountFactor=4&resultOffs

In [44]:
target_dates = [pd.to_datetime(date, unit='ms').strftime("%Y-%m-%d") for date in target_timestamps]
target_dates

['2022-03-31',
 '2022-04-30',
 '2022-05-31',
 '2022-06-30',
 '2022-07-31',
 '2022-08-31',
 '2022-09-30',
 '2022-10-31',
 '2022-11-30',
 '2022-12-31',
 '2023-01-31',
 '2023-02-28',
 '2023-03-31',
 '2023-04-30',
 '2023-05-31',
 '2023-06-30',
 '2023-07-31',
 '2023-08-31',
 '2023-09-30',
 '2023-10-31',
 '2023-11-30',
 '2023-12-31',
 '2024-01-31',
 '2024-02-29',
 '2024-03-31',
 '2024-04-30',
 '2024-05-31',
 '2024-06-30',
 '2024-07-31',
 '2024-08-31',
 '2024-09-30',
 '2024-10-31',
 '2024-11-30',
 '2024-12-31']

In [29]:
map_data

In [ ]:
dates[

Timestamp('2024-12-31 00:00:00')

In [8]:
response.content

b'<html lang="en">\r\n<head>\r\n  <title>Query: AssessedRussianAdvance_Merge45 (ID: 75)</title>\r\n  <link href=\'/ESRI.ArcGIS.SDS.REST.css\' rel=\'stylesheet\' type=\'text/css\'>\r\n  <link rel=\'SHORTCUT ICON\' href=\'/favicon.ico\'>\r\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8"/> \r\n</head>\r\n<body>\r\n<table width="100%" id="userTable" class="userTable">\r\n<tr>\r\n<td class="titlecell">ArcGIS REST Services Directory</td>\r\n</tr>\r\n</table>\r\n<table id="navTable" class="navTable" width="100%">\r\n<tbody>\r\n<tr valign="top">\r\n<td class="breadcrumbs">\r\n<span id="homeBreadcrumbs"><a href="/SaBe5HMtmnbqSWlu/ArcGIS/rest/services">Home</a> > <a href="/SaBe5HMtmnbqSWlu/ArcGIS/rest/services">services</a> </span>> <a href="/SaBe5HMtmnbqSWlu/ArcGIS/rest/services/UkrainianCoTTimelapse_FEB_2022_to_DEC_2024_view/FeatureServer">UkrainianCoTTimelapse_FEB_2022_to_DEC_2024_view (FeatureServer)</a> > <a href="/SaBe5HMtmnbqSWlu/ArcGIS/rest/services/UkrainianCoTTimela